# Single BERT Sample

> https://medium.com/towards-artificial-intelligence/text-classification-with-simple-transformers-a29d13358135



In [1]:
!pip install --upgrade transformers
!pip install simpletransformers

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.2.1)
     |████████████████████████████████| 102kB 6.3MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74842 sha256=68fc6bde121e8c42c37cdb83fa519b6530fdf4a6bde18f32d03b8d5596dde526
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=e5ee9ee50d27ab96e48d1bdcfe4d7200603be29b1a8fda55f4798917cbfc12f0
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog pathtools
  Found existing installation: watchdog 1.0.2
    Uninstalling watchdog-1.0.2:
      Successfully uninstalled watchdog-1.0.2


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
import logging
from google.colab import files
import json
import gc
import time

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [5]:
def sigmoid(x):
    z = (1/(1 + np.exp(-x)))
    return z

def show_confusion_matrix(confusion_matrix, name_model, file_matriz_name):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
    plt.title('Matriz de Confusão: ' + str(name_model))
    plt.ylabel('Classificação Real')
    plt.xlabel('Classificação Predita')
    plt.savefig(file_matriz_name)
    plt.close()

In [6]:
class arguments:  
    def __init__(self, model_name, batch_size, learning_rate, epochs, model_type):
        self.model_name = model_name
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.model_type = model_type

list_arguments = []

ARR_BATCH_SIZE = [16, 32]
ARR_LEARNING_RATE = [5e-5, 3e-5, 2e-5]
ARR_EPOCHS = [2, 3, 4, 5]
#ARR_EPOCHS_MODEL_DEFAULT = [2, 3, 5, 6, 7, 10]

I_MODEL = 0

for batch_size in ARR_BATCH_SIZE:
  for lr in ARR_LEARNING_RATE:
    for epoch in ARR_EPOCHS:
      I_MODEL += 1
      model_name = str("Model_"+str(I_MODEL))
      model_type = 'Fine_Tuning'
      list_arguments.append(arguments(model_name, batch_size, lr, epoch, model_type))

#for epoch in ARR_EPOCHS_MODEL_DEFAULT:
  #I_MODEL += 1
  #model_name = str("Model_"+str(I_MODEL))
  #model_type = 'Default'
  #list_arguments.append(arguments(model_name, 0, 0, epoch, model_type))

for params in list_arguments: 
    print('Modelo: ' + params.model_name, ' | Batch Size: ' + str(params.batch_size), ' | Learning Rate: ' + str(params.learning_rate), ' | Épocas: ' + str(params.epochs), sep = ' ' )

SEED=42
class_names = ['negativa', 'positiva']

Modelo: Model_1  | Batch Size: 16  | Learning Rate: 5e-05  | Épocas: 2
Modelo: Model_2  | Batch Size: 16  | Learning Rate: 5e-05  | Épocas: 3
Modelo: Model_3  | Batch Size: 16  | Learning Rate: 5e-05  | Épocas: 4
Modelo: Model_4  | Batch Size: 16  | Learning Rate: 5e-05  | Épocas: 5
Modelo: Model_5  | Batch Size: 16  | Learning Rate: 3e-05  | Épocas: 2
Modelo: Model_6  | Batch Size: 16  | Learning Rate: 3e-05  | Épocas: 3
Modelo: Model_7  | Batch Size: 16  | Learning Rate: 3e-05  | Épocas: 4
Modelo: Model_8  | Batch Size: 16  | Learning Rate: 3e-05  | Épocas: 5
Modelo: Model_9  | Batch Size: 16  | Learning Rate: 2e-05  | Épocas: 2
Modelo: Model_10  | Batch Size: 16  | Learning Rate: 2e-05  | Épocas: 3
Modelo: Model_11  | Batch Size: 16  | Learning Rate: 2e-05  | Épocas: 4
Modelo: Model_12  | Batch Size: 16  | Learning Rate: 2e-05  | Épocas: 5
Modelo: Model_13  | Batch Size: 32  | Learning Rate: 5e-05  | Épocas: 2
Modelo: Model_14  | Batch Size: 32  | Learning Rate: 5e-05  | Épocas: 3
M

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

file_path = "/content/drive/My Drive/data_full.csv"

data = pd.read_csv(file_path, sep=';').replace({np.NaN: None})
data.columns = ['sentenca','entidade1','entidade1_tipo','relacao','entidade2','entidade2_tipo']

cols = ['entidade1', 'entidade2']
data['entidades'] = data[cols].apply(lambda row: ' , '.join(row.values.astype(str)), axis=1)

data['class'] = data['relacao'].apply(lambda x: 0 if x is None else 1)

data.head()

Mounted at /content/drive


,sentenca,entidade1,entidade1_tipo,relacao,entidade2,entidade2_tipo,entidades,class
0,World Alliance of Reformed Churches condena a ...,Iraque,LOC,condena a guerra no,World Alliance of,ORG,"Iraque , World Alliance of",1
1,Em 19 de Dezembro de 1974 foi assinado um acor...,PAIGC,ORG,acordo entre,Portugal,LOC,"PAIGC , Portugal",1
2,Em 19 de Dezembro de 1974 foi assinado um acor...,PAIGC,ORG,None,Cabo Verde,LOC,"PAIGC , Cabo Verde",0
3,Em 19 de Dezembro de 1974 foi assinado um acor...,Portugal,LOC,None,Cabo Verde,LOC,"Portugal , Cabo Verde",0
4,"A 88i , plataforma de serviços digitais para s...",88i,ORG,foi aprovada no,Global Startup Program,ORG,"88i , Global Startup Program",1


In [ ]:
data['sentence'] = data[['sentenca', 'entidades']].apply(lambda row: ' , '.join(row.values.astype(str)), axis=1)
data = data[['sentence','class']]
data.head()

,sentence,class
0,World Alliance of Reformed Churches condena a ...,1
1,Em 19 de Dezembro de 1974 foi assinado um acor...,1
2,Em 19 de Dezembro de 1974 foi assinado um acor...,0
3,Em 19 de Dezembro de 1974 foi assinado um acor...,0
4,"A 88i , plataforma de serviços digitais para s...",1


In [ ]:
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['class'], random_state=SEED)
test_df, valid_df = train_test_split(test_df, test_size=0.5, stratify=test_df['class'], random_state=SEED)

print("train_df:", train_df.shape)
print("test_df:", test_df.shape)
print("valid_df:", valid_df.shape)

train_df: (2630, 2)
test_df: (329, 2)
valid_df: (329, 2)


In [ ]:
arr_file_names = []
model_args = ClassificationArgs()

file_train_dataset_name = str("train_dataset.csv")
file_validation_dataset_name = str("validation_dataset.csv")
file_test_dataset_name = str("test_dataset.csv")

arr_file_names.append(file_train_dataset_name)
arr_file_names.append(file_validation_dataset_name)
arr_file_names.append(file_test_dataset_name)

In [ ]:
for params in list_arguments: 
  if params.model_type == 'Fine_Tuning':
    print('Modelo: ' + params.model_name, ' | Batch Size: ' + str(params.batch_size), ' | Learning Rate: ' + str(params.learning_rate), ' | Épocas: ' + str(params.epochs), sep = ' ' )
    model_args.manual_seed = SEED
    model_args.learning_rate = params.learning_rate
    model_args.train_batch_size = params.batch_size
    model_args.num_train_epochs = params.epochs
    model_args.overwrite_output_dir = True
    model_args.save_steps = -1
    model_args.save_model_every_epoch = False
  else:
    print('Modelo: ' + params.model_name, ' | Épocas: ' + str(params.epochs), sep = ' ' )
    model_args.manual_seed = SEED
    model_args.num_train_epochs = params.epochs
    model_args.overwrite_output_dir = True
    model_args.save_steps = -1
    model_args.save_model_every_epoch = False

  model = ClassificationModel(
    model_type='bert', 
    model_name='neuralmind/bert-large-portuguese-cased', 
    use_cuda=True, 
    args=model_args
  )

  file_metrics_name = str("metrics_")+params.model_name+str(".json")
  file_matriz_name = str("matriz_")+params.model_name+str(".png")
  file_predictions_name = str("predictions_")+params.model_name+str(".csv")
  file_full_args_name = str("full_args_")+params.model_name+str(".txt")

  arr_file_names.append(file_metrics_name)
  arr_file_names.append(file_matriz_name)
  arr_file_names.append(file_predictions_name)
  arr_file_names.append(file_full_args_name)
  
  print(">>>>> " + str(params.model_name) + ": Treinando modelo")
  model.train_model(train_df)
  print(">>>>> " + str(params.model_name) + ": Validando com dados de Validação")
  result, model_outputs, wrong_preds = model.eval_model(valid_df)

  print(">>>>> " + str(params.model_name) + ": Calculando F-Score de Validação")
  sigmoid_preds = []
  predictions = []

  for x in model_outputs:
      sigmoid_pred = sigmoid(x)
      sigmoid_preds.append(np.argmax(sigmoid_pred))
      predictions.append(np.argmax(x))

  sigmoid_preds = np.array(sigmoid_preds)

  print('>>>>>>>>>> Arg-Max F1-score Validation:', round(f1_score(valid_df['class'], predictions), 4))
  print('>>>>>>>>>> Sigmoide F1-score Validation:', round(f1_score(valid_df['class'], sigmoid_preds), 4))

  print(">>>>> " + str(params.model_name) + ": Predizendo valores")
  test_df.reset_index(drop=True, inplace=True)
  test_predictions, raw_outputs = model.predict(test_df['sentence'])

  print(">>>>> " + str(params.model_name) + ": Métricas")
  print(classification_report(test_df['class'], test_predictions, target_names=class_names))

  report = classification_report(test_df['class'], test_predictions, target_names=class_names, output_dict=True)
  report['F1_score_Test'] = round(f1_score(test_df['class'], test_predictions), 4)
  report['F1_score_Validation'] = round(f1_score(valid_df['class'], predictions), 4)

  print(">>>>> " + str(params.model_name) + ": Salvando Métricas em JSON")
  
  with open(file_metrics_name, 'w') as outfile:
    json.dump(report, outfile)
  
  print(">>>>> " + str(params.model_name) + ": Gerando matriz de confusão e salvando")
  cnf_matrix = confusion_matrix(test_df['class'], test_predictions)
  df_cm = pd.DataFrame(cnf_matrix, index=class_names, columns=class_names)

  show_confusion_matrix(df_cm, str(params.model_name) + ' - ' + str(params.model_type), file_matriz_name)

  print(">>>>> " + str(params.model_name) + ": Salvando Arquivo com predições de Dados de Teste")
  df_final = pd.DataFrame(columns=['sentence','class','predicted_class'])
  df_final['sentence'] = test_df['sentence']
  df_final['class'] = test_df['class']
  df_final['predicted_class'] = test_predictions

  df_final.to_csv(file_predictions_name, sep=';', encoding='utf-8-sig') 
  
  text_file = open(file_full_args_name, "w")
  text_file.write(str(model_args))
  text_file.close()

  print(">>>>>>>>>>>>>>> " + str(params.model_name) + ": Fim")

INFO:filelock:Lock 140462844925936 acquired on /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5.lock


Modelo: Model_1  | Batch Size: 16  | Learning Rate: 5e-05  | Épocas: 2


Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

INFO:filelock:Lock 140462844925936 released on /root/.cache/huggingface/transformers/c534071830642050813fa94003dbf1234413b3f1d5dc66d259fbc82ff7d5fd59.c8340a82acfbbcd2dd960b86d2886ee120b21896ef0294150f0391918ae6ced5.lock
INFO:filelock:Lock 140463756874752 acquired on /root/.cache/huggingface/transformers/016fb7702039667c9fb9dd2ceffaf04027b13e525a6248cda2a4a87dbb8687af.881d7200bce807f871637ac9d552c541b2d4b00146a0bf1ab0360f3640031273.lock


Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

...

INFO:simpletransformers.classification.classification_model:{'mcc': 0.5715791238418152, 'tp': 121, 'tn': 137, 'fp': 44, 'fn': 27, 'auroc': 0.8641555920561446, 'auprc': 0.8317049612413894, 'eval_loss': 0.5162001549975858}
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


>>>>> Model_24: Calculando F-Score de Validação
>>>>>>>>>> Arg-Max F1-score Validation: 0.7732
>>>>>>>>>> Sigmoide F1-score Validation: 0.7732
>>>>> Model_24: Predizendo valores


  0%|          | 0/329 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

>>>>> Model_24: Métricas
              precision    recall  f1-score   support

    negativa       0.79      0.72      0.75       180
    positiva       0.69      0.77      0.73       149

    accuracy                           0.74       329
   macro avg       0.74      0.74      0.74       329
weighted avg       0.74      0.74      0.74       329

>>>>> Model_24: Salvando Métricas em JSON
>>>>> Model_24: Gerando matriz de confusão e salvando
>>>>> Model_24: Salvando Arquivo com predições de Dados de Teste
>>>>>>>>>>>>>>> Model_24: Fim


In [ ]:
train_df.to_csv(file_train_dataset_name, sep=';', encoding='utf-8-sig') 
valid_df.to_csv(file_validation_dataset_name, sep=';', encoding='utf-8-sig') 
test_df.to_csv(file_test_dataset_name, sep=';', encoding='utf-8-sig') 

for file in arr_file_names:
  files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
architecture_file_name = "architecture_model.txt"
text_file = open(architecture_file_name, "w")
text_file.write(str(model.model))
text_file.close()

files.download(architecture_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>